In [4]:
# packages
import os
import numpy as np
import pandas as pd
import pickle
import ray 
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import IFrame
from joblib import Parallel, delayed
fomr multiprocessing import 
from src.model_2state import solve_prep, solve_pre_jump_2state
from src.simulation_2d import simulation_2d, simulate_logkapital
from src.plots import plot_2S_ey1y2, plot_1S_vs_2S_ems, plot_1S_vs_2S_SCC, plot_2S_ey1y2_multi_λ, plot_1S_vs_2S_ems_multi_λ, plot_1S_vs_2S_SCC_multi_λ 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams["figure.edgecolor"] = "w"
mpl.rcParams['figure.facecolor'] = "w"
mpl.rcParams["savefig.dpi"] = 300

# Appendix C: when $Y$ has two states:

Process:

$$
dY_t^1  = Y_t^2 dt
$$

$$
dY_t^2 =   - \lambda Y_t^2 dt + \lambda \theta \mathcal{E} dt
$$


## Step I: post jump HJB:

The post jump HJB for $\phi_m(y_1, y_2)$, $m = 1, \dots, M$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}} \min_{\omega_\ell } & - \delta \phi_m(y_1, y_2) + \eta log(\mathcal{E}) \\
& + \frac{\partial \phi_m}{\partial y_1} y_2 + \frac{\partial \phi_m}{\partial y_2} \lambda (- y_2 + \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} \left(\gamma_1 + \gamma_2 y_1 + \gamma_3 (y_1 - \bar y)\mathbb{I}\{y_1>\bar y\} \right) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

First order condition for $\omega_\ell$, $\ell = 1, \dots, L$:

$$
    \omega_\ell \propto \pi_\ell^a \exp\left( -\frac{1}{\xi_a} \frac{\partial \phi_m}{\partial y_2}\lambda \theta_\ell \mathcal{E} \right), for \ell = 1, \dots, L
$$

and the first order condition for emission is:
$$
\mathcal{E} = - \cfrac{\eta}{\frac{\partial \phi_m }{\partial y_2} \lambda \sum_{\ell=1}^{L} \omega_\ell \theta_\ell}
$$


## Step II: pre jump HJB:

Given post jump value functions $\phi_m(y_1, y_2)$, $m = 1, 2, M$, solve the following HJB for pre-jump value function $\Phi(y_1, y_2)$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)\\
& + \mathcal{J}(y_1) \sum_{m=1}^M g_m \pi_d^m ( \phi_m(\bar{y}_1, y_2) - \Phi(y_1, y_2)) \\
& + \xi_p \mathcal{J}(y_1) \sum_{m=1}^M \pi_d^m \left(1 - g_m + g_m \log (g_m)\right)
\end{aligned}
$$

Or solve the following HJB with a terminal condition:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

$$
\Phi(\bar y_1, y_2) \approx  - \xi_p \log \left (\sum_{m=1}^M \pi_m^p \exp\left[-\frac{1}{\xi_p }\phi_m(\bar y_1, y_2) \right] \right) 
$$

In what follows, we show emission, $Y$ behavior before temperature anomay reaches the upper bound of jump threshold, $2^o C$, conditioning on no jump.

The $\lambda$ we choose here are:
- $\lambda = 0.116$, corresponds to half life of six years;
- $\lambda = 1, 2, 5$. Here, we try to illustrate emission behavior as $\lambda$ increase.

## Example 1: Half life of six years

In [5]:
λ = np.log(2) / 6
# parameters
δ = 0.01
η = 0.032
ξ_a = 0.01
θ_list = pd.read_csv("./data/model144.csv", header=None)[0].to_numpy()
θ_list = θ_list/1000
θ = np.mean(θ_list)
σy = 1.2*θ
# damage function
y_bar = 2.
γ_1 = 0.00017675
γ_2 = 2*0.0022
γ_3_list = np.linspace(0., 1./3, 20)
ξ_r = 1.
# %%
# y_grid
y1_step = .04
y1_grid = np.arange(0., 4. + y1_step, y1_step)

y2_step = .001
y2_grid = np.arange(0., .05 + y2_step, y2_step)
(y1_mat, y2_mat) = np.meshgrid(y1_grid,y2_grid, indexing = 'ij')

stateSpace = np.hstack([y1_mat.reshape(-1,1,order = 'F'),y2_mat.reshape(-1,1,order = 'F')])



In [6]:

args_list  = []
for γ_3_i in γ_3_list:
    args_iter = (y1_grid, y2_grid, γ_3_i, θ_list, (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 1., 1000, 0.05)
    args_list.append(args_iter)

if not os.path.exists(f"./data/res_list_{λ}"):
    with Pool() as p:
        res_list = p.starmap(solve_prep, args_list)

    with open(f"./data/res_list_{λ}", "wb") as handle:
        pickle.dump(res_list, handle)

# step II HJB:
with open(f"./data/res_list_{λ}", "rb") as file:
    res_list = pickle.load(file)

if not os.path.exists(f"./data/res_{λ}_{ξ_r}"):
    args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=0.1)
    with open(f"./data/res_{λ}_{ξp}", "wb") as handle:
        pickle.dump(res, handle)

NameError: name 'Pool' is not defined

In [7]:
#load results
res = pickle.load(open(f"./data/res_{λ}_{ξ_r}", "rb"))

In [8]:
et_prejump, y1t_prejump, y2t_prejump = simulation_2d(res,
                                                  θ=np.mean(θ_list),
                                                  y1_0 = 1.1,
                                                  y2_0=np.mean(θ_list),
                                                  T=110
                                                 )

In [9]:
simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
}

pickle.dump(simul, open(f"data/simul_{λ}", "wb"))

In [14]:
plot_2S_ey1y2(simul)


Compare the emission trajectories with the one state model:

In [15]:
et_1state = np.load("et_1state_cal.npy")
plot_1S_vs_2S_ems(et_1state, simul)


In [16]:
invkap = 0.09
α = 0.115
αₖ = - 0.043
σₖ = 0.0095
κ = 6.667
k0 = 85/α


In [17]:
Kt = simulate_logkapital(invkap, αₖ, σₖ, κ,  k0, T=111)
MC = δ*(1-η)/((α - invkap)*np.exp(Kt))
scc = η*(α - invkap)*np.exp(Kt)/(1-η)/et_prejump*1000
scc_1 = η*(α - invkap)*np.exp(Kt[:len(et_1state)])/(1-η)/et_1state*1000

In [20]:
# TODO: scc comparison
plot_1S_vs_2S_SCC(et_1state, scc, scc_1)


# Example 2: increasing $\lambda$

For the purpose of illustration, we choose $\lambda = 1, 2, 5$ to show what happens if the half life decreases.

In [21]:
res_dict = {}
λ_list = [1., 2., 5.]

for λ in λ_list:
    if not os.path.exists(f"./data/res_list_{λ}"):
        res_list = [delayed(solve_prep)(y1_grid, y2_grid, γ_3_i, θ_list, 
                                      (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 0.05, 10000, 0.05) for γ_3_i in γ_3_list]

        with open(f"./data/res_list_{λ}", "wb") as handle:
            pickle.dump(res_list, handle)

    # step II HJB:
    with open(f"./data/res_list_{λ}", "rb") as file:
        res_list = pickle.load(file)
    args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=.1)

    with open(f"./data/res_{λ}_{ξ_r}", "wb") as handle:
        pickle.dump(res, handle)
    res_dict[λ] = res

TypeError: tuple indices must be integers or slices, not str

In [22]:
simul_list = {}
for λ in λ_list:
    res_temp=pickle.load(open(f"./data/res_{λ}_{ξ_r}", "rb"))
    et_prejump, y1t_prejump, y2t_prejump = simulation_2d(res_temp,
                                                  θ=np.mean(θ_list),
                                                  y1_0 = 1.1,
                                                  y2_0=np.mean(θ_list),
                                                  T=110
                                                 )
    simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
    }
    simul_list[λ] = simul

pickle.dump(simul, open(f"data/simul_{λ}", "wb"))

In [24]:
# plot, emission, y1, y2, button, λ = 1, 2, 5
plot_2S_ey1y2_multi_λ(simul_list, λ_list)


In [26]:
#  emission, λ = 1,2,5 and 1 state result
plot_1S_vs_2S_ems_multi_λ(et_1state, simul_list, λ_list)


In [27]:
scc_list=np.zeros((3, 111))
for i in range(len(λ_list)):
    scc_list[i] = η*(α - invkap)*np.exp(Kt)/(1-η)/simul_list[λ_list[i]]["et"]*1000

In [28]:
# scc λ = 1, 2, 5 and 1 state
plot_1S_vs_2S_SCC_multi_λ(et_1state, scc_list, scc_1, λ_list)